# Multi-Learn Dreambooth

This tool is meant for automation of creating models of a person or multiple people
NOTE: This does NOT create a single model that knows all faces, but seperate models where ONE person is known
This tool simply make the process of making multiple models a lot easier.

In [ ]:
#!apt update
#!apt -y install git
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1

In [ ]:
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-person_ddim.git
!mkdir -p outputs/txt2img-samples/samples/person_ddim
!mv -v Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/*.* outputs/txt2img-samples/samples/person_ddim


In [ ]:

!gdown https://drive.google.com/uc?id=1LQUnD_v9iFx66XBTyZR9AkXWzgTiCQ6R -O model.ckpt

In [ ]:
!mkdir training_images

In [ ]:
models_to_train = [
    "ralvholmsen"
]

In [ ]:
import os

for new_model in models_to_train:
    project_name = new_model
    
    with open("ldm/data/__init__.py", "w") as f:
        f.write(f'word_to_train = "{new_model}" ')
    

    # MAX STEPS
    # It's how long you want your training to go.
    # If you're seeing this message, I'm literally at my computer right now fixing this up:
    max_training_steps = 1000

    class_word = "person"  # << match this word to the class word from regularization images above
    reg_data_root = "/workspace/Dreambooth-Stable-Diffusion-Multi-Learn/outputs/txt2img-samples/samples/person_ddim"
    new_sample_dir = "/workspace/Dreambooth-Stable-Diffusion-Multi-Learn/training_images/" + new_model+"/*"

#    !rm -rf "/workspace/Dreambooth-Stable-Diffusion-Multi-Learn/training_samples/*"
    
    !rm -rf "/workspace/Dreambooth-Stable-Diffusion-Multi-Learn/training_samples"
    !mkdir "/workspace/Dreambooth-Stable-Diffusion-Multi-Learn/training_samples"
    
    !cp {new_sample_dir} "/workspace/Dreambooth-Stable-Diffusion-Multi-Learn/training_samples/"
    
    !python "main.py" \
        --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
        -t \
        --actual_resume "model.ckpt" \
        --reg_data_root {reg_data_root} \
        -n {project_name} \
        --gpus 0, \
        --data_root "/workspace/Dreambooth-Stable-Diffusion-Multi-Learn/training_samples" \
        --max_training_steps {max_training_steps} \
        --class_word class_word \
        --no-test
    
    directory_paths = !ls -d logs/*
    last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"

    if os.path.exists(last_checkpoint_file):
        !python "prune_ckpt.py" --ckpt {last_checkpoint_file}

        last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
        training_samples = !ls training_samples
        date_string = !date +"%Y-%m-%dT%H-%M-%S"

        file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_samples)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + class_word + "_class_word.ckpt"
        !mkdir -p trained_models
        !mv {last_checkpoint_file_pruned} trained_models/{file_name}
        #!cp trained_models/{file_name} ../stable-diffusion-webui/models/stable-diffusion/{new_model}